## 自然语言统计

In [1]:
import random
import sys,os
sys.path.append(os.path.abspath("../"))
import d2l

/Users/zane/miniforge3/envs/torch/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 根据文本预处理中介绍的time_machine数据集构建Vocab,打印前10个最常用的单词
tokens = d2l.tokenize(d2l.read_time_machine())
coupus = [token for line in tokens for token in line]
vocab = d2l.Vocab(coupus)
vocab.token_freqs[:10]

[('the', 2261),
 ('i', 1267),
 ('and', 1245),
 ('of', 1155),
 ('a', 816),
 ('to', 695),
 ('was', 552),
 ('in', 541),
 ('that', 443),
 ('my', 440)]

词频最高的词往往都是很无聊 被称为 Stop Word,停用词,可以过滤掉.

## 读取长序列数据
序列数据本质上是连续的.当序列变得过长无法一次性全部处理时候,我们希望拆分这样的序列方便读取

### 随机采样
在随机采样中,每个样本都是在原始长序列上任意捕获的子序列,在迭代过程中,来自两个相邻的 随机的小批量的字序列不一定在原始序列上相邻.

In [3]:
import torch
# 下面代码每次可以从数据中随机生成一个小批量,在这里 参数batch_size制定了每个小批量中子序列样本的数目,参数num_steps是每个字序列中预定义 的时间步数.
def seq_data_iter_random(corpus, batch_size, num_steps):
    """使用随机抽样生成一个小批量子序列"""

    # 从随机偏移量开始对序列进行分区
    corpus = corpus[random.randint(0, num_steps - 1):]
    num_subseqs = (len(corpus) - 1) // num_steps

    initial_indices = list(range(0, num_subseqs * num_steps, num_steps))
    # 打乱每个子序列的起始点顺序
    random.shuffle(initial_indices)

    def data(position):
        return corpus[position:position + num_steps]

    num_batches = num_subseqs // batch_size
    for i in range(0,batch_size*num_batches,batch_size):
        initial_indices_perbatch = initial_indices[i:i+batch_size]
        X = [data(j) for j in initial_indices_perbatch]
        Y = [data(j + 1) for j in initial_indices_perbatch]
        yield torch.Tensor(X), torch.Tensor(Y)

In [4]:
my_seq = list(range(35))
for X,Y in seq_data_iter_random(my_seq, batch_size=2, num_steps=5):
    print(X,Y)

tensor([[11., 12., 13., 14., 15.],
        [16., 17., 18., 19., 20.]]) tensor([[12., 13., 14., 15., 16.],
        [17., 18., 19., 20., 21.]])
tensor([[21., 22., 23., 24., 25.],
        [26., 27., 28., 29., 30.]]) tensor([[22., 23., 24., 25., 26.],
        [27., 28., 29., 30., 31.]])
tensor([[ 1.,  2.,  3.,  4.,  5.],
        [ 6.,  7.,  8.,  9., 10.]]) tensor([[ 2.,  3.,  4.,  5.,  6.],
        [ 7.,  8.,  9., 10., 11.]])


In [1]:
def yield_test():
    for i in range(5):
        yield i
for i in yield_test():
    print(i)

0
1
2
3
4
